# ETL_EDA    
This file records the process of acquiring raw data, traforming them, and loading them into a MongoDB. The data are store (almost) in their raw form.

## 1. Raw Data from LAPD     
The [dataset](https://data.lacity.org/A-Safe-City/Arrest-Data-from-2010-to-Present/yru6-6re4) is a continuously updated file in csv format. It contains City of Los Angeles arrest data from 2010 to present. It can be retrieved simply by making requests without parameters. For more complicated API call, you may need to add query parameters.

In [22]:
import logging
import pandas as pd
from sodapy import Socrata
import utils

# download data
CRIME_SOURCE = "data.lacity.org"
client0 = Socrata(CRIME_SOURCE, None)
results = client0.get("yru6-6re4",where="arst_date > '2018-12-01T00:00:00.000'",limit = 1000000) # wait to be confirmed
print(results[:2])
print('...')
print(results[-2:])

[{'rpt_id': '5486720', 'arst_date': '2018-12-02T00:00:00.000', 'time': '1630', 'area': '12', 'area_desc': '77th Street', 'rd': '1259', 'age': '27', 'sex_cd': 'M', 'descent_cd': 'H', 'chrg_grp_cd': '04', 'grp_description': 'Aggravated Assault', 'arst_typ_cd': 'F', 'charge': '273.5(A)PC', 'chrg_desc': 'CORPORAL INJURY ON SPOUSE/COHABITANT/ETC', 'location': '300 E  74TH                         ST', 'location_1': {'latitude': '33.973', 'longitude': '-118.2695', 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}}, {'rpt_id': '5486572', 'arst_date': '2018-12-02T00:00:00.000', 'time': '0645', 'area': '13', 'area_desc': 'Newton', 'rd': '1391', 'age': '59', 'sex_cd': 'M', 'descent_cd': 'H', 'chrg_grp_cd': '14', 'grp_description': 'Sex (except rape/prst)', 'arst_typ_cd': 'M', 'charge': '647(A)PC', 'chrg_desc': 'SOLICIT/ENGAGE LEWD ACT IN PUBLIC VIEW', 'location': '69TH', 'crsst': 'GRAND', 'location_1': {'latitude': '33.9774', 'longitude': '-118.2803', 'human_address': '{"add

## 2. Raw Data to Documents/Dicts      
Datetime conversion is made and age column is changed to numeric data type. Now the data can be easily converted to a list of dicts which is what we want for the MongoDB.

In [23]:
from datetime import datetime

df = pd.DataFrame.from_records(results)
df['arst_date'] = pd.to_datetime(df['arst_date'])
# df['hour'] = df['time'].apply(lambda x: datetime.strptime(x, '%H%M').time().hour)
#df['month'] = df['arst_date'].apply(lambda x: datetime.strptime(x, '%Y%m').time())
df['age'] = pd.to_numeric(df['age'])
df.head()

,rpt_id,arst_date,time,area,area_desc,rd,age,sex_cd,descent_cd,chrg_grp_cd,grp_description,arst_typ_cd,charge,chrg_desc,location,location_1,crsst
0,5486720,2018-12-02,1630,12,77th Street,1259,27,M,H,04,Aggravated Assault,F,273.5(A)PC,CORPORAL INJURY ON SPOUSE/COHABITANT/ETC,300 E 74TH ST,"{'latitude': '33.973', 'longitude': '-118.2695...",NaN
1,5486572,2018-12-02,0645,13,Newton,1391,59,M,H,14,Sex (except rape/prst),M,647(A)PC,SOLICIT/ENGAGE LEWD ACT IN PUBLIC VIEW,69TH,"{'latitude': '33.9774', 'longitude': '-118.280...",GRAND
2,5486874,2018-12-02,2324,08,West LA,0814,53,M,H,22,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,SAN DIEGO FY,"{'latitude': '34.0728', 'longitude': '-118.466...",SUNSET BL
3,5486747,2018-12-02,1820,18,Southeast,1844,53,M,B,05,Burglary,F,459PC,BURGLARY,400 E LANZIT AV,"{'latitude': '33.9314', 'longitude': '-118.267...",NaN
4,5486478,2018-12-02,0445,12,77th Street,1273,32,M,B,12,Weapon (carry/poss),F,29800(A)1PC,POSS F/ARM BY CONVICTED FELON/ADDICT/ETC,MANCHESTER AV,"{'latitude': '33.96', 'longitude': '-118.309',...",WESTERN AV


## 3. Upsert MongoDB         
To avoid duplicate data entries between each run, the de-duplication mechanism makes sure that only a new record with a different report_id (rpt_id) will be inserted to the database. 

In [24]:
import pymongo
client = pymongo.MongoClient()

In [ ]:
db = client.get_database("crime")
collection = db.get_collection("crime")
update_count = 0
for record in df.to_dict('records'):
    result = collection.replace_one(
        filter = {'rpt_id': record['rpt_id']},    # locate the document if exists
        replacement = record,                         # latest document
        upsert=True)
    if result.matched_count > 0:
        update_count += 1
print(f"rows={df.shape[0]}, update={update_count}, "
      f"insert={df.shape[0]-update_count}")